<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Unidad_3/Viviendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `CLASIFICACIÓN DE VIVIENDA POR PRECIOS.`
# Armando Chávez Hernández.
**Geografía CUCSH.**

In [ ]:


import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import stats
#from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
#from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Cargar los datos
datos_vivienda = pd.read_csv('/content/drive/My Drive/Colab Data/house_class.csv')
display(datos_vivienda.head())
#Ubicación espacial

datos_vivienda.plot(kind='scatter', x='Lon', y='Lat', s=20, alpha=.5, color="blue")

In [ ]:
#EXPLORAR DATOS

print("Tamaño dataframe:", datos_vivienda.shape)
print("Presencia de lineas sin datos:", datos_vivienda.isnull().values.any())
print("Máximo de cuartos:", datos_vivienda.loc[:,['Room']].max().item())
print("Promedio de área:", round(datos_vivienda.loc[:, ['Area']].mean().item(),1))
print("Precio únicos:", datos_vivienda.loc[:, ['Price']].nunique().item())
print("Zip_area únicos:", datos_vivienda.loc[:, ['Zip_area']].nunique().item())
print("Zip_loc únicos:", datos_vivienda.loc[:, ['Zip_loc']].nunique().item())


display(datos_vivienda[['Area','Room','Lon','Lat']].describe())

In [ ]:
#VISUALIZACIONES DE DATOS

from matplotlib import pyplot as plt

levels_p, categories_p = pd.factorize(datos_vivienda['Price'])
plt.figure(figsize=(8, 6))
scatter_p = plt.scatter(x='Lon', y='Lat', data=datos_vivienda,
                      c=levels_p, s=32, alpha=.5)
plt.title('Precio')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend(scatter_p.legend_elements()[0], categories_p, title='Precio')
plt.show()

levels_a, categories_a = pd.factorize(datos_vivienda['Zip_area'])
scatter_a = plt.scatter(x='Lon', y='Lat', data=datos_vivienda,
                      c=levels_a, s=32, alpha=.8)
plt.legend(scatter_a.legend_elements()[0], categories_a, title='Zip_area')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.title('Patron de Zip_area')
plt.show()

datos_vivienda['Area'].plot(kind='hist', bins=20, title='Area')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.show()

datos_vivienda['Room'].plot(kind='hist', bins=20, title='Room')
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.show()

In [ ]:
#PREPARAR DATOS PARA CLASIFICACIÓN

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

X = datos_vivienda.loc[:, ['Area','Room','Lon','Lat','Zip_area','Zip_loc']]
y = datos_vivienda.loc[:, ['Price']]

enc = OrdinalEncoder()
enc.fit(X[['Zip_area', 'Zip_loc']])

X_transformed = pd.DataFrame(enc.transform(X[['Zip_area', 'Zip_loc']]),
                                       index=X.index)

X_full = X[['Area', 'Room', 'Lon', 'Lat']].join(X_transformed)

X_train, X_test, y_train, y_test = train_test_split(X_full.values, y.values,
                         stratify=X['Zip_loc'].values,
                         train_size=0.7, random_state=1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



In [ ]:
#ENTRENAR Y EVALUAR MODELO

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

dtree = DecisionTreeClassifier(criterion='entropy', max_features=3, splitter='best',
                                     max_depth=6, min_samples_split=4, random_state=3)
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)

print("Precisión: {}".format(round(report['accuracy'], 3)))
print("F1 score: {}".format(round(report['macro avg']['f1-score'], 3)))

#VISUALIZAR PREDICCIÓND E DATOS DEL MODELO

y_pred_full = dtree.predict(X_full.values)
levels_predict, categories_predict = pd.factorize(y_pred_full)

plt.figure(figsize=(8, 6))
scatter_predict = plt.scatter(x='Lon', y='Lat', data=datos_vivienda,
                      c=levels_predict, s=32, alpha=.5)
plt.title('Patron de precios en la predicción del modelo')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend(scatter_predict.legend_elements()[0], categories_predict, title='Precio')
plt.show()


In [ ]:
#VISUALIZAR LA DIFERENCIA ENTRE EL MODELO Y VALORES REALES, GRADO DE PRECISIÓN

y_pred_full = dtree.predict(X_full.values)
levels_predict, categories_predict = pd.factorize(y_pred_full)

plt.figure(figsize=(8, 6))
scatter_predict = plt.scatter(x='Lon', y='Lat', data=datos_vivienda,
                      c=levels_predict, s=32, alpha=.5)
plt.title('Patron de precios en la predicción del modelo')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend(scatter_predict.legend_elements()[0], categories_predict, title='Precio')
plt.show()
